In [0]:
%%time
from google.colab import drive
drive.mount('/content/drive')
%cd "drive/My Drive/pytorch/評価"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive/pytorch/評価
CPU times: user 435 ms, sys: 150 ms, total: 585 ms
Wall time: 21.8 s


In [0]:
%%time
! npm install -g localtunnel
!python3 -m pip install visdom
!python3 -m visdom.server -port 8076 >> visdomlog.txt 2>&1 &
!lt --port 8076 >> url.txt 2>&1 &

/tools/node/bin/lt -> /tools/node/lib/node_modules/localtunnel/bin/lt.js
+ localtunnel@2.0.0
added 35 packages from 21 contributors in 1.825s
     |████████████████████████████████| 686kB 3.5MB/s 
     |████████████████████████████████| 204kB 47.3MB/s 
  Created wheel for visdom: filename=visdom-0.1.8.9-cp36-none-any.whl size=655252 sha256=f1d0842e87982eaf25ab7a2c24ef14b6ed23543efce1c46aa1feddb1deb68856
  Stored in directory: /root/.cache/pip/wheels/70/19/a7/6d589ed967f4dfefd33bc166d081257bd4ed0cb618dccfd62a
  Created wheel for torchfile: filename=torchfile-0.1.0-cp36-none-any.whl size=5711 sha256=5f8deaf75508990312474ca4a0a9b6afbe407001c78bc852f1e339f7670ca2e3
  Stored in directory: /root/.cache/pip/wheels/b1/c3/d6/9a1cc8f3a99a0fc1124cae20153f36af59a6e683daca0a0814
Successfully built visdom torchfile
CPU times: user 243 ms, sys: 28.5 ms, total: 271 ms
Wall time: 23.7 s


In [0]:
%%time
import time
time.sleep(5)
! cat url.txt

CPU times: user 13.6 ms, sys: 5.05 ms, total: 18.6 ms
Wall time: 5.98 s


In [0]:
from lib import network_finetuning as network
from lib import architecture
from lib import dataloader as dl
from lib import optimizer
from lib import trainer

import torchvision

In [0]:
%%time
net = network.ResNet50(class_size=10)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [0]:
%%time
batch_size = 128
transform = dl.pattern_transform(resize=224, HorizontalFlip=True, VerticalFlip=False, Rotation=True, Perspective=True, Crop=True, Erasing=True)
#transform = dl.pattern_transform(resize=224, HorizontalFlip=True, VerticalFlip=True, Rotation=True, Perspective=True, Crop=False, Erasing=True)
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=dl.val_transform())
data_loader = dl.DataLoader(train_dataset, test_dataset, batch_size=batch_size,suffle=True, num_workers=2)

  0%|          | 0/170498071 [00:00<?, ?it/s]

[Resize(size=(224, 224), interpolation=PIL.Image.BILINEAR), RandomHorizontalFlip(p=0.5), RandomRotation(degrees=(-180, 180), resample=False, expand=False), RandomPerspective(p=0.5), RandomResizedCrop(size=(224, 224), scale=(0.08, 1.0), ratio=(0.75, 1.3333), interpolation=PIL.Image.BILINEAR), ToTensor(), <torchvision.transforms.transforms.RandomErasing object at 0x7f91e6328630>, Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))]


170500096it [00:01, 85969379.59it/s]                               


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
CPU times: user 3.28 s, sys: 998 ms, total: 4.28 s
Wall time: 6.64 s


In [0]:
%%time
model = architecture.CNN_Architecture(net)

CPU times: user 0 ns, sys: 39.4 ms, total: 39.4 ms
Wall time: 74.4 ms


In [0]:
%%time
model.train(data_loader, epoch_count=50)

cuda


Setting up a new session...


----------
Epoch 1/50 2020-01-29 04:45:59.458356
val Loss: 4.8176 Acc: 0.0994
----------
Epoch 2/50 2020-01-29 04:46:29.066955
train Loss: 2.1031 Acc: 0.2304
val Loss: 2.3600 Acc: 0.2528
----------
Epoch 3/50 2020-01-29 04:55:42.846922
train Loss: 1.9501 Acc: 0.2772
val Loss: 1.9386 Acc: 0.3103
----------
Epoch 4/50 2020-01-29 05:04:59.147100
train Loss: 1.9213 Acc: 0.2938
val Loss: 1.9093 Acc: 0.3101
----------
Epoch 5/50 2020-01-29 05:14:14.822110
train Loss: 1.8840 Acc: 0.3075
val Loss: 1.7782 Acc: 0.3535
----------
Epoch 6/50 2020-01-29 05:23:30.555989
train Loss: 1.8342 Acc: 0.3300
val Loss: 1.7694 Acc: 0.3614
----------
Epoch 7/50 2020-01-29 05:32:45.931450
train Loss: 1.7977 Acc: 0.3458
val Loss: 1.7607 Acc: 0.3690
----------
Epoch 8/50 2020-01-29 05:42:01.871267
train Loss: 1.7703 Acc: 0.3595
val Loss: 1.7166 Acc: 0.3766
----------
Epoch 9/50 2020-01-29 05:51:18.487415
train Loss: 1.7388 Acc: 0.3732
val Loss: 1.6965 Acc: 0.3983
----------
Epoch 10/50 2020-01-29 06:00:34.816794


In [0]:
%%time
model.predict(data_loader['val'])

検証画像に対しての正解率： 69 %
CPU times: user 19.5 s, sys: 12.5 s, total: 31.9 s
Wall time: 34 s
